# Firestore main 

<image src="figures/firestore1.png" width=300>

In [22]:
from google.cloud import firestore
db = 'test1'
db = firestore.Client.from_service_account_json('credentials.json', database=db)

**Plase note** that here the service account is required- However, if we use firestore in a flask application deployed in Google App Engine (PaaS) the service account is no longer needed, because the application is automatically able to access the firestore service.

*db = firestore.Client(database=db)*

### Entity Creation (Document)

In [11]:
coll = 'coll_test'
id = 'document1'
doc_ref = db.collection(coll).document(id) # id can be omitted and it will be generated automatically
doc_ref.set({'nome':'sensor','value': [{'val':2}]})
print(doc_ref.get().id)

document1


### Entity Removal

In [8]:
id = 'document1'
doc_ref = db.collection(coll).document(id)
doc_ref.delete()

DatetimeWithNanoseconds(2025, 4, 3, 11, 29, 48, 959712, tzinfo=datetime.timezone.utc)

### Access Entity given id


**Attention** To access a document you have to use a the document itself (entity) not the reference to a document (doc_ref). This is why there is a *.get()* at the end of the second line

In [12]:
# accesso a un documento specifico (dato l'id)
id = 'document1'
entity = db.collection(coll).document(id).get()
print(entity.exists)
print(entity.to_dict())
if not entity.exists:
    print('L\' entità non esiste')
else:
    print(list(entity.to_dict().keys()))
    print(entity.id,'--->',entity.to_dict()['nome'])
    print(entity.id,'--->',entity.to_dict()['value'])
    print(1 + entity.to_dict()['value'][0]['val'])

True
{'value': [{'val': 2}], 'nome': 'sensor'}
['value', 'nome']
document1 ---> sensor
document1 ---> [{'val': 2}]
3


### Modify Entity

Add an element or modify an element in the dictionary. 
The same code of the creation of an entiry allows to change the whole dictionary

**Attention** To modify a document you have to use a reference to the document (doc_ref) not the document itself (entity)


In [13]:
id = 'document1'
doc_ref = db.collection(coll).document(id)
doc_ref.update({'nome':'sensor2'})
doc_ref.update({'value2':5})


update_time {
  seconds: 1743679802
  nanos: 601350000
}

Increment 

In [14]:
id = 'document1'
doc_ref = db.collection(coll).document(id)
doc_ref.update({'value2': doc_ref.get().to_dict()['value2']+1})
doc_ref.update({'value2': firestore.Increment(50)})
print(doc_ref.get().to_dict()['value2'])

56


### Queries

In [15]:
for entity in db.collection(coll).stream(): # select * from coll_test
        print(f'{entity.id} --> {entity.to_dict()}')


document1 --> {'value': [{'val': 2}], 'nome': 'sensor2', 'value2': 56}


In [21]:
from google.cloud.firestore import FieldFilter

for entity in db.collection(coll).where('value2','>',100).stream():
#for entity in db.collection(coll).where(filter=FieldFilter("value2", ">", 100)).stream():

    
    print(f'{entity.id} --> {entity.to_dict()}')


c:\Users\marco\Dropbox\Code\PYTHON\PCLOUD25\venv\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:303: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
